## Dependencies

In [1]:
import re
import warnings
import pandas as pd
from joblib import load
import tensorflow_hub as hub
from tensorflow.keras import Model
from tensorflow.keras.layers import Lambda, Input, Dense, Dropout, Concatenate, Average, BatchNormalization, Activation
from googleqa_utilityscript import *

SEED = 0
seed_everything(SEED)
warnings.filterwarnings("ignore")

## Load data

In [2]:
module_url = '/kaggle/input/universalsentenceencodermodels/universal-sentence-encoder-models/use'
model_path = '/kaggle/input/48-googleq-a-train-use-skip/use.h5'
category_encoder_path = '/kaggle/input/48-googleq-a-train-use-skip/category_encoder.joblib'
scaler_path = '/kaggle/input/48-googleq-a-train-use-skip/scaler.joblib'

test = pd.read_csv('/kaggle/input/google-quest-challenge/test.csv')

print('Test samples: %s' % len(test))
display(test.head())

Test samples: 476


,qa_id,question_title,question_body,question_user_name,question_user_page,answer,answer_user_name,answer_user_page,url,category,host
0,39,Will leaving corpses lying around upset my pri...,I see questions/information online about how t...,Dylan,https://gaming.stackexchange.com/users/64471,There is no consequence for leaving corpses an...,Nelson868,https://gaming.stackexchange.com/users/97324,http://gaming.stackexchange.com/questions/1979...,CULTURE,gaming.stackexchange.com
1,46,Url link to feature image in the portfolio,I am new to Wordpress. i have issue with Featu...,Anu,https://wordpress.stackexchange.com/users/72927,I think it is possible with custom fields.\n\n...,Irina,https://wordpress.stackexchange.com/users/27233,http://wordpress.stackexchange.com/questions/1...,TECHNOLOGY,wordpress.stackexchange.com
2,70,"Is accuracy, recoil or bullet spread affected ...","To experiment I started a bot game, toggled in...",Konsta,https://gaming.stackexchange.com/users/37545,You do not have armour in the screenshots. Thi...,Damon Smithies,https://gaming.stackexchange.com/users/70641,http://gaming.stackexchange.com/questions/2154...,CULTURE,gaming.stackexchange.com
3,132,Suddenly got an I/O error from my external HDD,I have used my Raspberry Pi as a torrent-serve...,robbannn,https://raspberrypi.stackexchange.com/users/17341,Your Western Digital hard drive is disappearin...,HeatfanJohn,https://raspberrypi.stackexchange.com/users/1311,http://raspberrypi.stackexchange.com/questions...,TECHNOLOGY,raspberrypi.stackexchange.com
4,200,Passenger Name - Flight Booking Passenger only...,I have bought Delhi-London return flights for ...,Amit,https://travel.stackexchange.com/users/29089,I called two persons who work for Saudia (tick...,Nean Der Thal,https://travel.stackexchange.com/users/10051,http://travel.stackexchange.com/questions/4704...,CULTURE,travel.stackexchange.com


In [3]:
question_target_cols = ['question_asker_intent_understanding','question_body_critical', 'question_conversational', 
                        'question_expect_short_answer', 'question_fact_seeking', 'question_has_commonly_accepted_answer',
                        'question_interestingness_others', 'question_interestingness_self', 'question_multi_intent', 
                        'question_not_really_a_question', 'question_opinion_seeking', 'question_type_choice',
                        'question_type_compare', 'question_type_consequence', 'question_type_definition', 
                        'question_type_entity', 'question_type_instructions', 'question_type_procedure',
                        'question_type_reason_explanation', 'question_type_spelling', 'question_well_written']
answer_target_cols = ['answer_helpful', 'answer_level_of_information', 'answer_plausible', 'answer_relevance',
                      'answer_satisfaction', 'answer_type_instructions', 'answer_type_procedure', 
                      'answer_type_reason_explanation', 'answer_well_written']
target_cols = question_target_cols + answer_target_cols

## Pre-process data

In [4]:
text_features = ['question_title', 'question_body', 'answer']

for feature in text_features:
    # Lower
    test[feature] = test[feature].apply(lambda x: x.lower())
    # Map misspellings
#     test[feature] = test[feature].apply(lambda x: map_misspellings(x))
    # Map contractions
    test[feature] = test[feature].apply(lambda x: map_contraction(x))
    # Trim text
    test[feature] = test[feature].apply(lambda x: x.strip())
    
# Text length
test['question_title_len'] = test['question_title'].apply(lambda x : len(x))
test['question_body_len'] = test['question_body'].apply(lambda x : len(x))
test['answer_len'] = test['answer'].apply(lambda x : len(x))

# Word count
test['question_title_word_cnt'] = test['question_title'].apply(lambda x : len(x.split(' ')))
test['question_body_word_cnt'] = test['question_body'].apply(lambda x : len(x.split(' ')))
test['answer_word_cnt'] = test['answer'].apply(lambda x : len(x.split(' ')))

# Unique word count
test['question_title_unique_word_cnt'] = test['question_title'].apply(lambda x : len(set(x.split(' '))))
test['question_body_unique_word_cnt'] = test['question_body'].apply(lambda x : len(set(x.split(' '))))
test['answer_unique_word_cnt'] = test['answer'].apply(lambda x : len(set(x.split(' '))))

# Misspelings count
misspelling_mapping_set = set(misspelling_mapping.keys())
test['question_title_misspelings_cnt'] = test['question_title'].apply(lambda x : len(set(x.split(' ')) & misspelling_mapping_set))
test['question_body_misspelings_cnt'] = test['question_body'].apply(lambda x : len(set(x.split(' ')) & misspelling_mapping_set))
test['answer_misspelings_cnt'] = test['answer'].apply(lambda x : len(set(x.split(' ')) & misspelling_mapping_set))

## Test set

In [5]:
ohe_category = load(category_encoder_path)
scaler = load(scaler_path)


# Test features
X_test_title = test['question_title']
X_test_body = test['question_body']
X_test_answer = test['answer']
X_test_category = ohe_category.transform(test[['category']]).toarray()
X_test_dense = test[['question_title_len', 'question_title_word_cnt', 'question_title_unique_word_cnt', 'question_title_misspelings_cnt', 
                     'question_body_len', 'question_body_word_cnt', 'question_body_unique_word_cnt', 'question_body_misspelings_cnt', 
                     'answer_len', 'answer_word_cnt', 'answer_unique_word_cnt', 'answer_misspelings_cnt']]

X_test_dense = scaler.transform(X_test_dense)
X_test = [X_test_title, X_test_body, X_test_answer, X_test_category, X_test_dense]

# Model parameters

In [6]:
EMBEDDDING_SIZE = 512
N_CLASS = len(target_cols)

# Model

In [7]:
use_embed = hub.load(module_url)

def USEEmbedding(x):
    return use_embed(tf.squeeze(tf.cast(x, tf.string)))

def encoder_block(input_layer):
    encoder = Lambda(USEEmbedding, output_shape=(EMBEDDDING_SIZE,))(input_layer)
    
    return encoder

In [8]:
# Title
input_title = Input(shape=(1,), dtype=tf.string, name='input_title')
encoded_title = encoder_block(input_title)

# Body
input_body = Input(shape=(1,), dtype=tf.string, name='input_body')
encoded_body = encoder_block(input_body)

# Answer
input_answer = Input(shape=(1,), dtype=tf.string, name='input_answer')
encoded_answer = encoder_block(input_answer)

# Category
input_category = Input(shape=(X_test_category.shape[1],), dtype=tf.float32, name='input_category')

# Dense features
input_dense = Input(shape=(X_test_dense.shape[1],), dtype=tf.float32, name='input_dense')


# Output
dense = Concatenate()([input_dense, input_category])
dense = Dropout(0.2)(dense)
dense = Dense(32, kernel_initializer='glorot_uniform')(dense)
dense = BatchNormalization()(dense)
dense = Activation('relu')(dense)
dense = Dropout(0.2)(dense)
dense = Dense(16, kernel_initializer='glorot_uniform')(dense)
dense = BatchNormalization()(dense)
dense = Activation('relu')(dense)
dense = Dropout(0.2)(dense)

title = Dropout(0.2)(encoded_title)
title = Dense(128, kernel_initializer='glorot_uniform')(encoded_title)
title = BatchNormalization()(title)
title = Activation('relu')(title)
title = Dropout(0.2)(title)

body = Dropout(0.2)(encoded_body)
body = Dense(128, kernel_initializer='glorot_uniform')(encoded_body)
body = BatchNormalization()(body)
body = Activation('relu')(body)
body = Dropout(0.2)(body)

answer = Dropout(0.2)(encoded_answer)
answer = Dense(128, kernel_initializer='glorot_uniform')(encoded_answer)
answer = BatchNormalization()(answer)
answer = Activation('relu')(answer)
answer = Dropout(0.2)(answer)

question = Concatenate()([title, body])
question = Dropout(0.2)(question)
question = Dense(128, kernel_initializer='glorot_uniform')(question)
question = BatchNormalization()(question)
question = Activation('relu')(question)
question = Dropout(0.2)(question)

x = Concatenate()([question, answer, dense, encoded_title, encoded_body, encoded_answer])
x = Dense(512, kernel_initializer='glorot_uniform')(x)
x = BatchNormalization()(x)
x = Activation('relu')(x)
x = Dropout(0.5)(x)

output = Dense(N_CLASS, activation='sigmoid', kernel_initializer='glorot_uniform', name='output')(x)
model = Model(inputs=[input_title, input_body, input_answer, input_category, input_dense], outputs=[output])

model.summary()

model.load_weights(model_path)

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_title (InputLayer)        [(None, 1)]          0                                            
__________________________________________________________________________________________________
input_body (InputLayer)         [(None, 1)]          0                                            
__________________________________________________________________________________________________
lambda (Lambda)                 (None, 512)          0           input_title[0][0]                
__________________________________________________________________________________________________
lambda_1 (Lambda)               (None, 512)          0           input_body[0][0]                 
______________________________________________________________________________________________

# Make predictions

In [9]:
Y_test = model.predict(X_test)

In [10]:
submission = pd.read_csv('/kaggle/input/google-quest-challenge/sample_submission.csv')
submission[target_cols] = Y_test
submission.to_csv("submission.csv", index=False)
display(submission.head())

,qa_id,question_asker_intent_understanding,question_body_critical,question_conversational,question_expect_short_answer,question_fact_seeking,question_has_commonly_accepted_answer,question_interestingness_others,question_interestingness_self,question_multi_intent,...,question_well_written,answer_helpful,answer_level_of_information,answer_plausible,answer_relevance,answer_satisfaction,answer_type_instructions,answer_type_procedure,answer_type_reason_explanation,answer_well_written
0,39,0.849998,0.602182,0.185895,0.708010,0.643677,0.719806,0.621519,0.595547,0.239614,...,0.852179,0.877211,0.598717,0.910942,0.937533,0.784415,0.095340,0.073591,0.756490,0.886245
1,46,0.864813,0.433917,0.002622,0.730351,0.860421,0.915312,0.540441,0.459927,0.240669,...,0.643903,0.928205,0.583464,0.958232,0.963421,0.808595,0.935740,0.062926,0.027424,0.816882
2,70,0.876877,0.577532,0.034049,0.774884,0.893934,0.972141,0.572262,0.470274,0.302466,...,0.805927,0.877909,0.586491,0.964867,0.952209,0.779833,0.117488,0.077867,0.780053,0.869560
3,132,0.846598,0.473651,0.012520,0.715795,0.680329,0.815728,0.542701,0.440950,0.241793,...,0.658910,0.909154,0.641040,0.957826,0.968004,0.824981,0.807521,0.090332,0.695455,0.842587
4,200,0.957804,0.535304,0.027665,0.859370,0.690806,0.845449,0.660066,0.648285,0.350448,...,0.812361,0.967589,0.706339,0.993220,0.993032,0.908340,0.106873,0.161448,0.859977,0.935285
